In [4]:
# --------------------------------------------------------------------------
# Phase 3.5: 데이터 검수 및 정제 (자동 검증 중심 - 수정된 스키마 적용)
# --------------------------------------------------------------------------
print("Starting Phase 3.5: Data Validation (JSON Format & Schema - Revised)")

# 필요한 라이브러리 (jsonschema는 필요시 설치: !pip install jsonschema)
import pandas as pd
import json
import os # 파일 존재 확인용
from tqdm.notebook import tqdm # 진행률 표시

try:
    from jsonschema import validate
    from jsonschema.exceptions import ValidationError
    JSONSCHEMA_AVAILABLE = True
except ImportError:
    print("Warning: jsonschema library not found. Skipping automatic schema validation.")
    print("Install it using: pip install jsonschema")
    JSONSCHEMA_AVAILABLE = False

# --- 설정 ---
INPUT_JSONL_PATH = "converted_data.jsonl" # 검증할 입력 파일
OUTPUT_JSONL_VALIDATED = "finetuning_data_validated.jsonl" # 검증 통과 데이터 저장 파일
# -------------

# --- 수정된 TARGET_JSON_SCHEMA 정의 ---
# 타입 이름을 표준 소문자로, nullable을 타입 리스트로 변경
TARGET_JSON_SCHEMA = {
    'type': 'object',
    'properties': {
        'attributes': {
            # attributes 객체 자체가 null일 수도 있음 (is_noise=True 경우 등)
            'type': ['object', 'null'],
            'description': "각 속성(맛, 식감, 가격, 포장/양, 재구매 의사)에 대한 분석 결과.",
            'properties': {
                '맛': {
                    'type': ['object', 'null'], # 속성 객체도 null 가능
                    'description': "맛 관련 언급 분석.",
                    'properties': {
                        'sentiment': {'type': ['string', 'null'], 'enum': ['positive', 'negative', 'mixed', 'neutral', None]},
                        'descriptors': {'type': ['array', 'null'], 'items': {'type': 'string'}}
                    },
                    # 이 객체 내 필수가 아님 (sentiment, descriptors 모두 null 가능)
                },
                '식감': {
                    'type': ['object', 'null'],
                    'description': "식감 관련 언급 분석.",
                    'properties': {
                        'sentiment': {'type': ['string', 'null'], 'enum': ['positive', 'negative', 'mixed', 'neutral', None]},
                        'descriptors': {'type': ['array', 'null'], 'items': {'type': 'string'}}
                    }
                },
                '가격': {
                    'type': ['object', 'null'],
                    'description': "가격 관련 언급 분석.",
                    'properties': {
                        'sentiment': {'type': ['string', 'null'], 'enum': ['positive', 'negative', 'neutral', None]},
                        'descriptors': {'type': ['array', 'null'], 'items': {'type': 'string'}}
                    }
                },
                 '포장_양': {
                    'type': ['object', 'null'],
                    'description': "포장 또는 양 관련 언급 분석.",
                    'properties': {
                        'sentiment': {'type': ['string', 'null'], 'enum': ['positive', 'negative', 'neutral', None]},
                        'descriptors': {'type': ['array', 'null'], 'items': {'type': 'string'}}
                    }
                 },
                '재구매': {
                   'type': ['object', 'null'],
                   'description': "재구매 의사 관련 언급 분석.",
                   'properties': {
                       'sentiment': {'type': ['string', 'null'], 'enum': ['positive', 'negative', 'neutral', None]},
                       'descriptors': {'type': ['array', 'null'], 'items': {'type': 'string'}}
                   }
                 }
            },
            # attributes 객체 자체는 필수가 아닐 수 있으므로 required 불필요
        },
        'meta': {
            'type': ['object', 'null'],
            'description': "댓글의 메타 정보.",
            'properties': {
                 '편의점언급': {'type': ['string', 'null']},
                 '제품명언급': {'type': ['string', 'null']}
            }
            # meta 객체 자체는 필수가 아닐 수 있으므로 required 불필요
        },
        'is_noise': {
            'type': 'boolean', # 필수 필드
            'description': "리뷰가 분석 대상이 아니면 true."
        },
        'overall_sentiment': {
            'type': ['string', 'null'], # 필수 필드지만 null 가능
            'enum': ['positive', 'negative', 'mixed', 'neutral', None],
            'description': "댓글 전체의 종합적인 감성."
        }
    },
    # 최상위 레벨 필수 필드 (null 허용 여부는 위에서 정의)
    'required': ['attributes', 'meta', 'is_noise', 'overall_sentiment']
}
# --- 스키마 정의 끝 ---

validated_records = []
valid_data_count = 0
invalid_json_count = 0
schema_violation_count = 0
total_lines = 0

print(f"Starting validation for file: '{INPUT_JSONL_PATH}'...")

# 입력 파일 존재 확인
if not os.path.exists(INPUT_JSONL_PATH):
    print(f"ERROR: Input file not found: '{INPUT_JSONL_PATH}'. Please ensure the file exists.")
else:
    try:
        with open(INPUT_JSONL_PATH, 'r', encoding='utf-8') as infile:
            # tqdm으로 파일 라인 읽기 진행 표시
            for i, line in enumerate(tqdm(infile, desc="Reading & Validating Lines")):
                total_lines += 1
                line = line.strip()
                if not line: continue # 빈 줄 스킵

                is_valid_record = True
                record = None
                output_json = None # 초기화

                # 1. 각 라인이 유효한 JSON 객체인지 확인 및 필수 키 검사
                try:
                    record = json.loads(line)
                    if 'text_input' not in record or 'output' not in record:
                        print(f"\n  - Line {i+1}: Missing 'text_input' or 'output' key. Skipping.")
                        invalid_json_count += 1
                        is_valid_record = False
                except json.JSONDecodeError as e:
                    print(f"\n  - Line {i+1}: Invalid JSON format. Error: {e}. Skipping line: {line[:100]}...")
                    invalid_json_count += 1
                    is_valid_record = False

                if not is_valid_record: continue # 다음 라인으로

                # 2. 'output' 값이 유효한 JSON 문자열인지 파싱 시도
                output_str = record.get('output')
                try:
                    # output 문자열을 Python 객체로 변환
                    output_json = json.loads(output_str)
                except (json.JSONDecodeError, TypeError):
                    print(f"\n  - Line {i+1}: The 'output' value is not a valid JSON string. Skipping. Value: {str(output_str)[:100]}...")
                    invalid_json_count += 1 # 이것도 JSON 오류로 카운트
                    is_valid_record = False

                if not is_valid_record: continue # 다음 라인으로

                # 3. 스키마 유효성 검증 (jsonschema 라이브러리가 있을 경우)
                if JSONSCHEMA_AVAILABLE:
                    try:
                        # 파싱된 output_json 객체를 스키마와 비교
                        validate(instance=output_json, schema=TARGET_JSON_SCHEMA)
                    except ValidationError as e:
                        # 스키마 위반 시 상세 정보 출력 (어떤 필드, 어떤 규칙 위반 등)
                        print(f"\n  - Line {i+1}: Schema violation in 'output' JSON. Skipping.")
                        print(f"    Violation details: {e.message[:200]}") # 오류 메시지 일부 표시
                        print(f"    Violating instance path: {list(e.path)}")
                        print(f"    Violating schema path: {list(e.schema_path)}")
                        schema_violation_count += 1
                        is_valid_record = False
                    except Exception as e_schema: # jsonschema 라이브러리 자체 오류 등
                        print(f"\n  - Line {i+1}: Unexpected error during schema validation: {e_schema}. Skipping.")
                        is_valid_record = False

                # 모든 검증을 통과한 경우에만 최종 리스트에 추가
                if is_valid_record:
                    validated_records.append(record) # 원본 record ({text_input, output}) 저장
                    valid_data_count += 1

    except Exception as e:
        print(f"\nError processing the input file: {e}")

    # 최종 결과 요약
    print(f"\nValidation Summary:")
    print(f"  - Total lines read: {total_lines}")
    print(f"  - Invalid JSON format / Missing keys: {invalid_json_count}")
    if JSONSCHEMA_AVAILABLE:
        print(f"  - Schema violations detected: {schema_violation_count}")
    else:
         print(f"  - Schema validation skipped (jsonschema not available).")
    print(f"  - 최종 유효 레코드 수: {valid_data_count}")

    # 검증 통과한 데이터 저장
    if validated_records:
        print(f"\nSaving {valid_data_count} validated records to '{OUTPUT_JSONL_VALIDATED}'...")
        try:
            with open(OUTPUT_JSONL_VALIDATED, 'w', encoding='utf-8') as outfile:
                for record in validated_records:
                    outfile.write(json.dumps(record, ensure_ascii=False) + '\n')
            print("Validated JSONL file saved successfully.")
            print("--- Phase 3.5: Data Validation Finished (Success) ---")
            FINAL_TRAINING_FILE_PATH = OUTPUT_JSONL_VALIDATED
        except Exception as e:
            print(f"Error saving validated JSONL file: {e}")
            print("--- Phase 3.5: Data Validation Finished (Save Failed) ---")
            FINAL_TRAINING_FILE_PATH = None
    else:
        print("\nNo valid data remaining after validation to save.")
        print("--- Phase 3.5: Data Validation Finished (No Valid Data) ---")
        FINAL_TRAINING_FILE_PATH = None

print("-" * 50)

Starting Phase 3.5: Data Validation (JSON Format & Schema - Revised)
Starting validation for file: 'converted_data.jsonl'...


Reading & Validating Lines: 0it [00:00, ?it/s]


  - Line 29: Schema violation in 'output' JSON. Skipping.
    Violation details: 'mixed' is not one of ['positive', 'negative', 'neutral', None]
    Violating instance path: ['attributes', '포장_양', 'sentiment']
    Violating schema path: ['properties', 'attributes', 'properties', '포장_양', 'properties', 'sentiment', 'enum']

  - Line 55: Schema violation in 'output' JSON. Skipping.
    Violation details: 'mixed' is not one of ['positive', 'negative', 'neutral', None]
    Violating instance path: ['attributes', '가격', 'sentiment']
    Violating schema path: ['properties', 'attributes', 'properties', '가격', 'properties', 'sentiment', 'enum']

  - Line 95: Schema violation in 'output' JSON. Skipping.
    Violation details: 'mixed' is not one of ['positive', 'negative', 'neutral', None]
    Violating instance path: ['attributes', '포장_양', 'sentiment']
    Violating schema path: ['properties', 'attributes', 'properties', '포장_양', 'properties', 'sentiment', 'enum']

  - Line 98: Schema violation i

In [7]:
# -*- coding: utf-8 -*-
# @title Step 1: Install and Import Necessary Libraries
# Install the required libraries: google-genai for the API and google-cloud-storage for GCS interactions (optional but good practice)
!pip install -U -q google-genai google-cloud-storage

print("Libraries installed successfully.")


from google import genai
from google.genai import types
import google.cloud.aiplatform as aiplatform
from google.api_core import exceptions as core_exceptions # For specific API error handling
from google.colab import userdata # To securely access secrets
import os
import time
import json # Useful for potentially validating JSON schema later

print(f"Using google-genai version: {genai.__version__}")

Libraries installed successfully.
Using google-genai version: 1.12.1


In [8]:
# @title Step 2: Load API Key from Colab Secrets
# Ensure you have stored your GOOGLE_API_KEY in Colab Secrets (Add-ons -> Secrets)
try:
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    if not GOOGLE_API_KEY:
        raise ValueError("API key not found. Please set the GOOGLE_API_KEY secret.")
    print("Successfully loaded GOOGLE_API_KEY.")
except userdata.SecretNotFoundError:
    print("ERROR: GOOGLE_API_KEY secret not found.")
    print("Please go to 'Add-ons' -> 'Secrets' and add your Google API Key as 'GOOGLE_API_KEY'.")
    # Stop execution if the key is not found
    raise SystemExit("API Key is required to proceed.")
except ValueError as e:
    print(f"ERROR: {e}")
    raise SystemExit("API Key is required to proceed.")

Successfully loaded GOOGLE_API_KEY.


In [9]:
# @title Step 3: Initialize the Gemini API Client
# Initialize the client using the API key, as recommended in the README
try:
    client = genai.Client(api_key=GOOGLE_API_KEY)
    print("Gemini API Client initialized successfully.")
except Exception as e:
    print(f"ERROR: Failed to initialize Gemini API Client: {e}")
    raise SystemExit("Client initialization failed.")

Gemini API Client initialized successfully.


In [10]:
# @title Step 4: Verify Setup by Listing Tunable Models
# List models that support fine-tuning to verify the client setup and identify potential base models
print("Checking for models that support fine-tuning ('createTunedModel')...")
available_tunable_models = []
try:
    for model in client.models.list():
        if "createTunedModel" in model.supported_actions:
            print(f"  - Found tunable model: {model.name} (Display Name: {model.display_name})")
            available_tunable_models.append(model.name)

    if not available_tunable_models:
        print("\nWARNING: No models supporting fine-tuning were found with the current API key/settings.")
        print("Please ensure your API key has the necessary permissions and that tunable models are available.")
    else:
        print("\nSetup seems correct. Found models capable of fine-tuning.")
        # Suggest a default base model based on findings or common choices
        default_base_model = next((m for m in available_tunable_models if 'gemini-1.5-flash' in m), available_tunable_models[0])
        print(f"Suggested base model for tuning: {default_base_model}")

except Exception as e:
    print(f"\nERROR: Failed to list models: {e}")
    print("This might indicate an issue with authentication or API access.")
    raise SystemExit("Model listing failed.")

Checking for models that support fine-tuning ('createTunedModel')...
  - Found tunable model: models/gemini-1.5-flash-001-tuning (Display Name: Gemini 1.5 Flash 001 Tuning)

Setup seems correct. Found models capable of fine-tuning.
Suggested base model for tuning: models/gemini-1.5-flash-001-tuning


In [13]:
# @title Step 5: Define GCS Configuration and Training Data Path
# --- USER INPUT REQUIRED ---
# Please replace the placeholder values below with your actual GCP Project ID, GCS Bucket Name,
# and the path to your JSONL training data file within the bucket.

GCP_PROJECT_ID = "snatch-456405"  # Replace with your Google Cloud Project ID (used for context, though not directly by Dev API tuning)
GCS_BUCKET_NAME = "bucket-quickstart_snatch-456405"  # Replace with your GCS bucket name (e.g., 'snatch-project-data')
GCS_JSONL_FILE_PATH = "finetuning_data_validated.jsonl" # Replace with the path inside your bucket (e.g., 'finetuning/reviews_train.jsonl')

# --- END OF USER INPUT ---

# Validate inputs (basic check)
if any(val.startswith("YOUR_") for val in [GCP_PROJECT_ID, GCS_BUCKET_NAME, GCS_JSONL_FILE_PATH]):
    print("ERROR: Please replace the placeholder values in the GCS Configuration section.")
    raise SystemExit("GCS Configuration is required.")

# Construct the full GCS URI for the training data
FINETUNING_GCS_URI = f"gs://{GCS_BUCKET_NAME}/{GCS_JSONL_FILE_PATH}"

print(f"Using GCP Project ID (for context): {GCP_PROJECT_ID}")
print(f"Using GCS Bucket: {GCS_BUCKET_NAME}")
print(f"Training Data GCS URI: {FINETUNING_GCS_URI}")


# to verify the GCS URI exists, but the tuning job will fail if it doesn't.
from google.cloud import storage
storage_client = storage.Client(project=GCP_PROJECT_ID) # May require separate auth if not using API key creds
try:
   blob = storage.Blob.from_string(FINETUNING_GCS_URI, client=storage_client)
   if not blob.exists():
       print(f"WARNING: The specified GCS URI does not seem to exist: {FINETUNING_GCS_URI}")
except Exception as e:
   print(f"WARNING: Could not verify GCS URI existence: {e}")

Using GCP Project ID (for context): snatch-456405
Using GCS Bucket: bucket-quickstart_snatch-456405
Training Data GCS URI: gs://bucket-quickstart_snatch-456405/finetuning_data_validated.jsonl


In [17]:
# @title Step 6: Define Fine-tuning Job Parameters (Modified for Inline Examples)
# --- USER INPUT REQUIRED ---
# Select the base model and set hyperparameters for the tuning job.

BASE_MODEL_ID = default_base_model # Or manually set: "models/gemini-1.5-flash-001-tuning"
TUNED_MODEL_DISPLAY_NAME = "SNATCH_Review_JSON_Converter_v1"
EPOCH_COUNT = 5
BATCH_SIZE = 8

# --- END OF USER INPUT ---

print(f"Base Model for Tuning: {BASE_MODEL_ID}")
print(f"Tuned Model Display Name: {TUNED_MODEL_DISPLAY_NAME}")
print(f"Epoch Count: {EPOCH_COUNT}")
print(f"Batch Size: {BATCH_SIZE}")

# --- NEW: Download and Prepare Data for Inline Examples ---
print("\nDownloading training data from GCS to Colab environment...")
from google.colab import auth
from google.cloud import storage
import json
import tempfile

local_jsonl_path = None
training_examples = [] # List to hold TuningExample objects

try:
    # Authenticate Colab user for GCS access
    auth.authenticate_user()
    storage_client = storage.Client(project=GCP_PROJECT_ID)
    bucket = storage_client.bucket(GCS_BUCKET_NAME)
    blob = bucket.blob(GCS_JSONL_FILE_PATH)

    if not blob.exists():
         raise FileNotFoundError(f"ERROR: GCS file not found at {FINETUNING_GCS_URI}")

    # Download to a temporary file
    with tempfile.NamedTemporaryFile(mode='wb', delete=False) as temp_file:
        local_jsonl_path = temp_file.name
        blob.download_to_filename(local_jsonl_path)
        print(f"Successfully downloaded GCS file to: {local_jsonl_path}")

    # Parse the downloaded JSONL file and create TuningExamples
    print("Parsing downloaded JSONL file...")
    count = 0
    with open(local_jsonl_path, 'r', encoding='utf-8') as f:
        for line in f:
            try:
                # Load each line as a JSON object
                data = json.loads(line.strip())
                # Ensure required keys exist
                if 'text_input' in data and 'output' in data:
                    # Create a TuningExample object
                    example = types.TuningExample(
                        text_input=data['text_input'],
                        output=data['output'] # Ensure this is the target JSON string
                    )
                    training_examples.append(example)
                    count += 1
                else:
                    print(f"WARNING: Skipping line due to missing 'text_input' or 'output': {line.strip()}")
            except json.JSONDecodeError:
                print(f"WARNING: Skipping invalid JSON line: {line.strip()}")
            except Exception as parse_err:
                 print(f"WARNING: Error processing line: {line.strip()} - {parse_err}")


    print(f"Successfully parsed {count} examples from the JSONL file.")

    if not training_examples:
        raise ValueError("ERROR: No valid training examples could be parsed from the file.")

    # Create the TuningDataset using the 'examples' list
    training_dataset = types.TuningDataset(examples=training_examples)
    print("Training dataset configured using inline examples.")


except FileNotFoundError as fnf_err:
    print(fnf_err)
    raise SystemExit("Cannot proceed without training data.")
except Exception as e:
    print(f"ERROR: Failed to download or parse training data: {e}")
    raise SystemExit("Data preparation failed.")
finally:
    # Clean up the temporary file
    if local_jsonl_path and os.path.exists(local_jsonl_path):
        os.remove(local_jsonl_path)
        print(f"Removed temporary file: {local_jsonl_path}")


# --- End of NEW section ---


# Define the tuning job configuration (remains the same)
tuning_config = types.CreateTuningJobConfig(
    tuned_model_display_name=TUNED_MODEL_DISPLAY_NAME,
    epoch_count=EPOCH_COUNT,
    batch_size=BATCH_SIZE,
)
print("Tuning job configuration created.")


# Define the target JSON schema (for reference, remains the same)
TARGET_JSON_SCHEMA = {
  "type": "object",
  "properties": {
    "attributes": {
      "type": ["object", None],
      "description": "각 속성(맛, 식감, 가격, 포장/양, 재구매 의사) 분석 결과.",
      "properties": {
        "맛":    {"type": ["object", None], "properties": {"sentiment": {"type": ["string", None], "enum": ["positive", "negative", "mixed", "neutral", None]}, "descriptors": {"type": ["array", None], "items": {"type": "string"}}}},
        "식감":  {"type": ["object", None], "properties": {"sentiment": {"type": ["string", None], "enum": ["positive", "negative", "mixed", "neutral", None]}, "descriptors": {"type": ["array", None], "items": {"type": "string"}}}},
        "가격":  {"type": ["object", None], "properties": {"sentiment": {"type": ["string", None], "enum": ["positive", "negative", "neutral", None]}, "descriptors": {"type": ["array", None], "items": {"type": "string"}}}},
        "포장_양": {"type": ["object", None], "properties": {"sentiment": {"type": ["string", None], "enum": ["positive", "negative", "neutral", None]}, "descriptors": {"type": ["array", None], "items": {"type": "string"}}}},
        "재구매": {"type": ["object", None], "properties": {"sentiment": {"type": ["string", None], "enum": ["positive", "negative", "neutral", None]}, "descriptors": {"type": ["array", None], "items": {"type": "string"}}}}
      }
    },
    "meta": {
      "type": ["object", None],
      "description": "메타 정보.",
      "properties": {
         "편의점언급": {"type": ["string", None]},
         "제품명언급": {"type": ["string", None]}
      }
    },
    "is_noise": {
      "type": "boolean",
      "description": "리뷰가 분석 대상이 아니면 true."
    },
    "overall_sentiment": {
      "type": ["string", None],
      "enum": ["positive", "negative", "mixed", "neutral", None]
    }
  },
  "required": ["attributes", "meta", "is_noise", "overall_sentiment"]
}
print("Target JSON schema defined (for reference). Ensure training data 'output' conforms to this.")

Base Model for Tuning: models/gemini-1.5-flash-001-tuning
Tuned Model Display Name: SNATCH_Review_JSON_Converter_v1
Epoch Count: 5
Batch Size: 8

Successfully downloaded GCS file to: /tmp/tmpxe7wcuvr
Parsing downloaded JSONL file...
Successfully parsed 103 examples from the JSONL file.
Training dataset configured using inline examples.
Removed temporary file: /tmp/tmpxe7wcuvr
Tuning job configuration created.
Target JSON schema defined (for reference). Ensure training data 'output' conforms to this.


In [18]:
# @title Step 7: Start and Monitor the Fine-tuning Job
print("Starting the fine-tuning job...")

tuning_op = None # Initialize operation variable
tuned_model_resource_name = None # To store the final model name

try:
    # Start the tuning job
    tuning_op = client.tunings.tune(
        base_model=BASE_MODEL_ID,
        training_dataset=training_dataset,
        config=tuning_config,
    )
    print(f"Tuning job started successfully. Operation Name: {tuning_op.name}")
    print("Monitoring job progress... This can take minutes to hours depending on data size and queue.")

except core_exceptions.InvalidArgument as e:
    print(f"\nERROR: Invalid argument provided to the tuning API: {e}")
    print("Please check base_model_id, GCS URI format, and tuning parameters.")
except core_exceptions.PermissionDenied as e:
    print(f"\nERROR: Permission denied: {e}")
    print("Ensure the API key has permissions for tuning and access to the GCS bucket.")
except Exception as e:
    print(f"\nERROR: An unexpected error occurred while starting the tuning job: {e}")
    tuning_op = None # Ensure we don't proceed if starting failed

# --- Monitoring Loop ---
if tuning_op:
    tuning_job_name = tuning_op.name
    start_time = time.time()
    polling_interval_seconds = 60 # Check status every minute

    while True:
        try:
            # Get the latest status of the tuning job
            tuning_job = client.tunings.get(name=tuning_job_name)
            current_state = tuning_job.state.name
            elapsed_time = time.time() - start_time
            print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] Job State: {current_state} (Elapsed: {elapsed_time:.0f}s)")

            if current_state == 'JOB_STATE_SUCCEEDED':
                # IMPORTANT: Access the tuned model information correctly
                # The structure might be tuning_job.tuned_model.model or tuning_job.tuned_model.endpoint
                # Check the actual response object structure from your SDK version / API docs
                if hasattr(tuning_job, 'tuned_model') and tuning_job.tuned_model:
                     # Prefer 'endpoint' if available, otherwise 'model' might contain the resource name
                    if hasattr(tuning_job.tuned_model, 'endpoint') and tuning_job.tuned_model.endpoint:
                         tuned_model_resource_name = tuning_job.tuned_model.endpoint
                    elif hasattr(tuning_job.tuned_model, 'model') and tuning_job.tuned_model.model:
                         tuned_model_resource_name = tuning_job.tuned_model.model
                    else:
                         print("WARNING: Could not determine the exact tuned model resource name from the response.")
                         tuned_model_resource_name = f"Check job details for: {tuning_job_name}"
                else:
                     print("WARNING: Tuned model details not found in the successful job response.")
                     tuned_model_resource_name = f"Check job details for: {tuning_job_name}"

                print("\n-----------------------------------------")
                print("🎉 Fine-tuning Job Succeeded! 🎉")
                print(f"Tuned Model Resource Name: {tuned_model_resource_name}")
                print(f"Original Job Name: {tuning_job_name}")
                print("You can now use this resource name in `client.models.generate_content()`")
                print("-----------------------------------------")
                break # Exit the monitoring loop

            elif current_state in ['JOB_STATE_FAILED', 'JOB_STATE_CANCELLED']:
                print("\n-----------------------------------------")
                print(f"❌ Fine-tuning Job {current_state}! ❌")
                if hasattr(tuning_job, 'error') and tuning_job.error:
                     # Attempt to parse error details if available
                    try:
                         error_details = json.dumps(tuning_job.error, indent=2)
                         print(f"Error Details:\n{error_details}")
                    except TypeError: # Handle cases where error might not be JSON serializable
                         print(f"Error Details: {tuning_job.error}")
                else:
                     print("No specific error details provided in the response.")
                print(f"Original Job Name: {tuning_job_name}")
                print("-----------------------------------------")
                break # Exit the monitoring loop

            elif current_state in ['JOB_STATE_PENDING', 'JOB_STATE_RUNNING']:
                # Job is still in progress, wait before polling again
                time.sleep(polling_interval_seconds)

            else:
                # Handle unexpected states
                print(f"\nWARNING: Encountered unexpected job state: {current_state}")
                print("Continuing to monitor...")
                time.sleep(polling_interval_seconds)

        except core_exceptions.NotFound:
            print(f"\nERROR: Tuning job '{tuning_job_name}' not found. It might have been deleted or the name is incorrect.")
            break
        except Exception as e:
            print(f"\nERROR: An error occurred while monitoring the job status: {e}")
            print("Stopping monitoring loop. Please check the job status manually in AI Studio or via API.")
            break # Exit on monitoring error
else:
    print("\nSkipping monitoring as the tuning job failed to start.")

# Final check
if tuned_model_resource_name:
    print(f"\nProcess complete. Use this model name for generation: {tuned_model_resource_name}")
else:
    print("\nProcess complete. Tuning job did not succeed or model name could not be retrieved.")

Starting the fine-tuning job...
Tuning job started successfully. Operation Name: tunedModels/snatchreviewjsonconverterv1-sbydql18q0f2
Monitoring job progress... This can take minutes to hours depending on data size and queue.
[2025-04-29 05:10:11] Job State: JOB_STATE_RUNNING (Elapsed: 0s)
[2025-04-29 05:11:11] Job State: JOB_STATE_RUNNING (Elapsed: 61s)
[2025-04-29 05:12:12] Job State: JOB_STATE_RUNNING (Elapsed: 121s)
[2025-04-29 05:13:12] Job State: JOB_STATE_RUNNING (Elapsed: 182s)
[2025-04-29 05:14:13] Job State: JOB_STATE_SUCCEEDED (Elapsed: 243s)

-----------------------------------------
🎉 Fine-tuning Job Succeeded! 🎉
Tuned Model Resource Name: tunedModels/snatchreviewjsonconverterv1-sbydql18q0f2
Original Job Name: tunedModels/snatchreviewjsonconverterv1-sbydql18q0f2
You can now use this resource name in `client.models.generate_content()`
-----------------------------------------

Process complete. Use this model name for generation: tunedModels/snatchreviewjsonconverterv1-sbyd

In [19]:
# @title Step 8: Test the Fine-tuned Model

import json # To potentially validate or pretty-print the JSON output

# Get the tuned model name from the previous step's output
# Ensure this variable holds the correct name printed at the end of Step 7
tuned_model_name = tuned_model_resource_name
# Or manually copy-paste:
# tuned_model_name = "tunedModels/snatchreviewjsonconverterv1-sbydql18q0f2"

if not tuned_model_name or not tuned_model_name.startswith("tunedModels/"):
     print("ERROR: `tuned_model_resource_name` is not set correctly. Please ensure Step 7 completed successfully.")
     # raise SystemExit("Tuned model name is required.") # Optional: Stop execution if name is missing
else:
    print(f"Using fine-tuned model: {tuned_model_name}\n")

    # --- 샘플 리뷰 텍스트 (한국어) ---
    # 여기에 테스트하고 싶은 새로운 리뷰들을 넣어보세요.
    sample_reviews = [
        "GS25 갔다가 새로 나온 크림빵 사봤는데 진짜 부드럽고 맛있네요! 우유 크림이 가득 차있어서 좋았어요. 가격도 2500원이면 괜찮은 편인듯. 또 사먹을 것 같아요.",
        "이 푸딩은 별로였음... 너무 달기만 하고 식감이 뚝뚝 끊어지는 느낌? 포장은 예쁜데 양도 적고 비싸기만 하네. 재구매 의사 없음.",
        "CU 신상 디저트! 쫀득한 떡 안에 팥이랑 크림치즈가 들어있는데 조합이 신선하고 맛있었다. 호불호는 갈릴 수 있을 듯? 나는 만족!",
        "걍 평범한 초코케이크 맛인데? 특별한 건 모르겠음. 가격 생각하면 쏘쏘.",
        "맛 괜찮은데 이거 인스턴트 면들이 해결하지 못하는 툭툭 끊어지고 기름발린면 느낌은 여전해서 아쉬웠음.", # 약간 노이즈성 또는 정보 부족 댓글
        "배고파서 아무거나 삼" # 완전 노이즈성 댓글
        "앗 전 너무 별로였어요 진짜는 안먹어봤지만 고무맛 났어요 ㅠㅠ 버렸어오ㅠㅠ"
        "일단 맛은 확실합니다. 즉 조미료 맛이나지만 잘하는 알리오 음식점 맛을 잘 구현해낸 인스턴트맛이에요.근데 좀 간이 있고 인스턴트의한계가 있다보니 라면 느낌이 나는건 어쩔수가"
    ]

    # --- 각 샘플 리뷰에 대해 예측 수행 ---
    for i, review in enumerate(sample_reviews):
        print(f"--- [Test Case {i+1}] ---")
        print(f"Input Review:\n{review}\n")

        try:
            # 튜닝된 모델을 사용하여 콘텐츠 생성 요청
            response = client.models.generate_content(
                model=tuned_model_name,
                contents=review
                # 참고: 튜닝된 모델은 일반적으로 별도의 system_instruction이나 복잡한 프롬프팅 없이
                # 입력 텍스트만으로 원하는 형식의 출력을 내도록 학습됩니다.
            )

            # 모델의 원시 출력 (JSON 문자열 형태여야 함)
            model_output_text = response.text
            print(f"Model Output (Raw Text):\n{model_output_text}\n")

            # --- 출력 검증 (JSON 형식인지 확인) ---
            try:
                # 모델 출력을 JSON 객체로 파싱 시도
                parsed_json = json.loads(model_output_text)
                print("JSON Validation: Success! Output is valid JSON.")

                # (선택) 파싱된 JSON을 보기 좋게 출력
                # print("\nParsed JSON Output:")
                # print(json.dumps(parsed_json, indent=2, ensure_ascii=False))

            except json.JSONDecodeError as json_err:
                print(f"JSON Validation: FAILED! Output is not valid JSON. Error: {json_err}")
            except Exception as e:
                print(f"An unexpected error occurred during JSON validation: {e}")

        except Exception as e:
            # API 호출 중 발생할 수 있는 일반적인 오류 처리
            print(f"ERROR: Failed to generate content for this review: {e}")

        print("-" * (20 + len(f"[Test Case {i+1}]")) + "\n")

    print("Finished testing all sample reviews.")

Using fine-tuned model: tunedModels/snatchreviewjsonconverterv1-sbydql18q0f2

--- [Test Case 1] ---
Input Review:
GS25 갔다가 새로 나온 크림빵 사봤는데 진짜 부드럽고 맛있네요! 우유 크림이 가득 차있어서 좋았어요. 가격도 2500원이면 괜찮은 편인듯. 또 사먹을 것 같아요.

Model Output (Raw Text):
{"attributes": {"맛": {"sentiment": "positive", "descriptors": ["진짜 부드럽고 맛있네요!", "우유 크림이 가득 차있어서 좋았어요"]}, "식감": {"sentiment": "positive", "descriptors": ["진짜 부드럽고"]}, "가격": {"sentiment": "neutral", "descriptors": ["가격도 2500원이면 괜찮은 편인듯"]}, "포장_양": {"sentiment": "positive", "descriptors": ["우유 크림이 가득 차있어서"]}, "재구매": {"sentiment": "positive", "descriptors": ["또 사먹을 것 같아요"]}}, "meta": {"편의점언급": "GS25", "제품명언급": "크림빵"}, "is_noise": false, "overall_sentiment": "positive"}

JSON Validation: Success! Output is valid JSON.
---------------------------------

--- [Test Case 2] ---
Input Review:
이 푸딩은 별로였음... 너무 달기만 하고 식감이 뚝뚝 끊어지는 느낌? 포장은 예쁜데 양도 적고 비싸기만 하네. 재구매 의사 없음.

Model Output (Raw Text):
아, 이 푸딩이 별로였군요.{"attributes": {"맛": {"sentiment": "negative", "descriptors": 

In [22]:
# @title Step 8: Test the Fine-tuned Model (Corrected - No Extra Params)

import json
import re # 정규 표현식 라이브러리 임포트 (후처리용)

tuned_model_name = tuned_model_resource_name
# tuned_model_name = "tunedModels/snatchreviewjsonconverterv1-sbydql18q0f2"

if not tuned_model_name or not tuned_model_name.startswith("tunedModels/"):
     print("ERROR: `tuned_model_resource_name` is not set correctly.")
     # raise SystemExit("Tuned model name is required.")
else:
    print(f"Using fine-tuned model: {tuned_model_name}\n").

    sample_reviews = [
        "GS25 갔다가 새로 나온 크림빵 사봤는데 진짜 부드럽고 맛있네요! 우유 크림이 가득 차있어서 좋았어요. 가격도 2500원이면 괜찮은 편인듯. 또 사먹을 것 같아요.",
        "이 푸딩은 별로였음... 너무 달기만 하고 식감이 뚝뚝 끊어지는 느낌? 포장은 예쁜데 양도 적고 비싸기만 하네. 재구매 의사 없음.",
        "CU 신상 디저트! 쫀득한 떡 안에 팥이랑 크림치즈가 들어있는데 조합이 신선하고 맛있었다. 호불호는 갈릴 수 있을 듯? 나는 만족!",
        "걍 평범한 초코케이크 맛인데? 특별한 건 모르겠음. 가격 생각하면 쏘쏘.",
        "맛 괜찮은데 이거 인스턴트 면들이 해결하지 못하는 툭툭 끊어지고 기름발린면 느낌은 여전해서 아쉬웠음.",
        "배고파서 아무거나 삼앗 전 너무 별로였어요 진짜는 안먹어봤지만 고무맛 났어요 ㅠㅠ 버렸어오ㅠㅠ일단 맛은 확실합니다. 즉 조미료 맛이나지만 잘하는 알리오 음식점 맛을 잘 구현해낸 인스턴트맛이에요.근데 좀 간이 있고 인스턴트의한계가 있다보니 라면 느낌이 나는건 어쩔수가"
    ]

    for i, review in enumerate(sample_reviews):
        print(f"--- [Test Case {i+1}] ---")
        print(f"Input Review:\n{review}\n")

        model_output_text = "" # 출력 초기화
        parsed_json = None # 파싱된 JSON 초기화
        validation_result = "FAILED" # 기본 상태는 실패로 설정

        try:
            # generate_content 호출 시 model과 contents만 사용
            response = client.models.generate_content(
                model=tuned_model_name,
                contents=review,
                # system_instruction 및 generation_config 제거됨
            )

            model_output_text = response.text
            print(f"Model Output (Raw Text):\n{model_output_text}\n")

            # --- 출력 검증 및 후처리 ---
            try:
                # 1. 원본 출력이 바로 JSON인지 시도
                parsed_json = json.loads(model_output_text)
                validation_result = "Success (Original)"
            except json.JSONDecodeError:
                # 2. 원본 출력이 JSON이 아니면, 후처리 시도 (첫 '{'와 마지막 '}' 사이 추출)
                print("Original output is not valid JSON. Attempting post-processing...")
                try:
                    # 정규 표현식으로 가장 바깥쪽 {} 사이의 내용 찾기 (더 견고함)
                    match = re.search(r'\{.*\}', model_output_text, re.DOTALL)
                    if match:
                        extracted_json_string = match.group(0)
                        print(f"Extracted JSON part:\n{extracted_json_string}\n")
                        parsed_json = json.loads(extracted_json_string)
                        validation_result = "Success (Post-processed)"
                    else:
                        print("Could not extract JSON part using regex.")
                        validation_result = "FAILED (Post-processing failed)"
                except json.JSONDecodeError as post_json_err:
                     print(f"Extracted part is not valid JSON. Error: {post_json_err}")
                     validation_result = "FAILED (Extracted part invalid)"
                except Exception as post_err:
                     print(f"Error during post-processing: {post_err}")
                     validation_result = f"FAILED (Post-processing error: {post_err})"


            print(f"JSON Validation Result: {validation_result}")
            if parsed_json:
                # (선택) 성공적으로 파싱된 경우 보기 좋게 출력
                print("\nParsed JSON:")
                print(json.dumps(parsed_json, indent=2, ensure_ascii=False))
                pass


        except Exception as e:
            print(f"ERROR: Failed to generate content for this review: {e}")

        print("-" * (20 + len(f"[Test Case {i+1}]")) + "\n")

    print("Finished testing all sample reviews.")

Using fine-tuned model: tunedModels/snatchreviewjsonconverterv1-sbydql18q0f2

--- [Test Case 1] ---
Input Review:
GS25 갔다가 새로 나온 크림빵 사봤는데 진짜 부드럽고 맛있네요! 우유 크림이 가득 차있어서 좋았어요. 가격도 2500원이면 괜찮은 편인듯. 또 사먹을 것 같아요.

Model Output (Raw Text):
{"attributes": {"맛": {"sentiment": "positive", "descriptors": ["진짜 부드럽고 맛있네요!", "우유 크림이 가득 차있어서 좋았어요"]}, "식감": {"sentiment": "positive", "descriptors": ["진짜 부드럽고"]}, "가격": {"sentiment": "neutral", "descriptors": ["가격도 2500원이면 괜찮은 편인듯"]}, "포장_양": {"sentiment": "positive", "descriptors": ["우유 크림이 가득 차있어서 좋았어요"]}, "재구매": {"sentiment": "positive", "descriptors": ["또 사먹을 것 같아요"]}}, "meta": {"편의점언급": "GS25", "제품명언급": "크림빵"}, "is_noise": false, "overall_sentiment": "positive"}

JSON Validation Result: Success (Original)

Parsed JSON:
{
  "attributes": {
    "맛": {
      "sentiment": "positive",
      "descriptors": [
        "진짜 부드럽고 맛있네요!",
        "우유 크림이 가득 차있어서 좋았어요"
      ]
    },
    "식감": {
      "sentiment": "positive",
      "descriptors": [
        "진짜 부

- 위 코드는 모델의 원시 출력을 그대로 '출력'만 하는 것이 아니라,
- 원시 출력을 일차적으로 '검증'하고,
- 검증에 실패하면 원시 출력에서 의심되는 JSON 부분을 '추출'하여 '가공'하고,
- 추출된 가공 결과물을 다시 '검증'한 후,
- 최종적으로 유효하다고 판단된 '가공된' 결과를 사용하고 출력

In [25]:
# -*- coding: utf-8 -*-
# @title Step 9: Process and Store Results in a Structured Format (Complete Code)

import json
import re
import pandas as pd # Pandas 라이브러리 임포트
import time # Timestamp 등을 위해 사용 가능

# 이전 Step 8의 tuned_model_resource_name 변수가 계속 사용된다고 가정합니다.
# tuned_model_name 변수가 이전 셀에서 설정되었는지 확인하세요.
# 예시: tuned_model_name = tuned_model_resource_name

# tuned_model_name 변수가 유효한지 확인
if 'tuned_model_name' not in globals() or not isinstance(tuned_model_name, str) or not tuned_model_name.startswith("tunedModels/"):
     print("ERROR: `tuned_model_name` is not set correctly or is not available.")
     print("Please ensure Step 7 completed successfully and the `tuned_model_resource_name` variable was correctly passed or copied.")
     raise SystemExit("Tuned model name is required to process results.")

print(f"Using fine-tuned model: {tuned_model_name}\n")

# --- 샘플 리뷰 텍스트 ---
# 테스트에 사용할 샘플 리뷰 목록 (이전과 동일)
# 실제 운영에서는 데이터베이스, 파일 또는 API 등 다른 소스에서 리뷰를 로드해야 합니다.
sample_reviews = [
    "GS25 갔다가 새로 나온 크림빵 사봤는데 진짜 부드럽고 맛있네요! 우유 크림이 가득 차있어서 좋았어요. 가격도 2500원이면 괜찮은 편인듯. 또 사먹을 것 같아요.",
    "이 푸딩은 별로였음... 너무 달기만 하고 식감이 뚝뚝 끊어지는 느낌? 포장은 예쁜데 양도 적고 비싸기만 하네. 재구매 의사 없음.",
    "CU 신상 디저트! 쫀득한 떡 안에 팥이랑 크림치즈가 들어있는데 조합이 신선하고 맛있었다. 호불호는 갈릴 수 있을 듯? 나는 만족!",
    "걍 평범한 초코케이크 맛인데? 특별한 건 모르겠음. 가격 생각하면 쏘쏘.",
    "맛 괜찮은데 이거 인스턴트 면들이 해결하지 못하는 툭툭 끊어지고 기름발린면 느낌은 여전해서 아쉬웠음.",
    "배고파서 아무거나 삼앗 전 너무 별로였어요 진짜는 안먹어봤지만 고무맛 났어요 ㅠㅠ 버렸어오ㅠㅠ일단 맛은 확실합니다. 즉 조미료 맛이나지만 잘하는 알리오 음식점 맛을 잘 구현해낸 인스턴트맛이에요.근데 좀 간이 있고 인스턴트의한계가 있다보니 라면 느낌이 나는건 어쩔수가"
]

# 처리된 결과를 저장할 리스트 초기화
processed_reviews_list = []
# 실패한 처리를 저장할 리스트 (선택 사항, 디버깅에 유용)
# failed_reviews_list = []

# --- 각 샘플 리뷰에 대해 예측 및 처리 수행 ---
print(f"Processing {len(sample_reviews)} sample reviews and structuring results...")

for i, review in enumerate(sample_reviews):
    review_id = f"review_{i+1}" # 각 리뷰를 식별할 수 있는 간단한 ID (실제는 DB primary key 등 사용)
    print(f"--- [Processing {review_id}/{len(sample_reviews)}] ---")
    # print(f"Input Review:\n{review}\n") # 리뷰 텍스트가 길 경우 출력 생략

    model_output_text = ""
    parsed_json = None
    validation_result = "FAILED" # 기본 상태는 실패

    try:
        # 튜닝된 모델 호출 (model과 contents만 사용, system_instruction 등 제거)
        response = client.models.generate_content(
            model=tuned_model_name,
            contents=review,
        )
        model_output_text = response.text
        # print(f"Model Output (Raw Text):\n{model_output_text}\n") # 모델 원시 출력 확인용 (필요시 주석 해제)

        # --- 출력 검증 및 후처리 ---
        try:
            # 1. 원본 출력이 바로 JSON인지 시도
            parsed_json = json.loads(model_output_text)
            validation_result = "Success (Original)"
        except json.JSONDecodeError:
            # 2. 원본 출력이 JSON이 아니면, 후처리 시도 (첫 '{'와 마지막 '}' 사이 추출)
            # print("Original output is not valid JSON. Attempting post-processing...") # 중간 출력 생략
            try:
                # 정규 표현식으로 가장 바깥쪽 {} 사이의 내용 찾기 (더 견고함)
                match = re.search(r'\{.*\}', model_output_text, re.DOTALL)
                if match:
                    extracted_json_string = match.group(0)
                    # print(f"Extracted JSON part:\n{extracted_json_string}\n") # 추출된 JSON 부분 확인용
                    parsed_json = json.loads(extracted_json_string)
                    validation_result = "Success (Post-processed)"
                else:
                    # print("Could not extract JSON part using regex.") # 중간 출력 생략
                    validation_result = "FAILED (Extraction failed)"
            except json.JSONDecodeError as post_json_err:
                 # print(f"Extracted part is not valid JSON. Error: {post_json_err}") # 중간 출력 생략
                 validation_result = "FAILED (Extracted part invalid)"
            except Exception as post_err:
                 # print(f"Error during post-processing: {post_err}") # 중간 출력 생략
                 validation_result = f"FAILED (Post-processing error: {post_err})"

        # print(f"JSON Validation Result: {validation_result}") # 최종 검증 결과 확인용 (필요시 주석 해제)

        # --- 성공적으로 파싱된 경우 데이터 구조화 ---
        if parsed_json is not None and validation_result.startswith("Success"):
            # attributes 객체가 없거나 null인 경우를 대비하여 안전하게 접근
            # .get()의 기본값 {}은 키가 없을 때 빈 딕셔너리를 반환.
            # 하지만 키는 있고 값이 null인 경우 get()은 None을 반환. 따라서 None 체크 필요.
            attributes_data = parsed_json.get("attributes", {})
            if attributes_data is None:
                attributes_data = {} # attributes가 명시적으로 null인 경우

            # meta 객체가 없거나 null인 경우를 대비하여 안전하게 접근
            meta_data = parsed_json.get("meta", {})
            if meta_data is None:
                meta_data = {} # meta가 명시적으로 null인 경우

            structured_data = {
                "review_id": review_id, # 리뷰 식별자 추가
                "original_review_text": review, # 원본 리뷰 텍스트
                "is_noise": parsed_json.get("is_noise", False), # is_noise 필드 (기본값 False)
                "overall_sentiment": parsed_json.get("overall_sentiment", None), # overall_sentiment 필드
                # meta 정보 플래트닝 (안전하게 접근)
                "meta_편의점언급": meta_data.get("편의점언급", None),
                "meta_제품명언급": meta_data.get("제품명언급", None),
                # attributes 정보 플래트닝 (각 속성에 대해 안전하게 접근)
                # 각 속성 객체(e.g., attributes_data.get("맛", {}))가 없거나 null인 경우를 대비
                "attr_맛_sentiment": attributes_data.get("맛", {}).get("sentiment", None),
                # descriptors가 null인 경우 .join()을 위해 빈 리스트 [] 사용
                "attr_맛_descriptors": ", ".join(attributes_data.get("맛", {}).get("descriptors", []) if attributes_data.get("맛", {}).get("descriptors") is not None else []),
                "attr_식감_sentiment": attributes_data.get("식감", {}).get("sentiment", None),
                "attr_식감_descriptors": ", ".join(attributes_data.get("식감", {}).get("descriptors", []) if attributes_data.get("식감", {}).get("descriptors") is not None else []),
                "attr_가격_sentiment": attributes_data.get("가격", {}).get("sentiment", None),
                "attr_가격_descriptors": ", ".join(attributes_data.get("가격", {}).get("descriptors", []) if attributes_data.get("가격", {}).get("descriptors") is not None else []),
                "attr_포장_양_sentiment": attributes_data.get("포장_양", {}).get("sentiment", None),
                "attr_포장_양_descriptors": ", ".join(attributes_data.get("포장_양", {}).get("descriptors", []) if attributes_data.get("포장_양", {}).get("descriptors") is not None else []),
                "attr_재구매_sentiment": attributes_data.get("재구매", {}).get("sentiment", None),
                "attr_재구매_descriptors": ", ".join(attributes_data.get("재구매", {}).get("descriptors", []) if attributes_data.get("재구매", {}).get("descriptors") is not None else []),
                # 후처리로 복구되었는지 여부 플래그 추가 (데이터 품질 추적에 유용)
                "is_post_processed": validation_result == "Success (Post-processed)",
                "processing_timestamp": pd.Timestamp.now() # 처리 타임스탬프 추가 (선택 사항)
            }
            processed_reviews_list.append(structured_data)
            print(f"--> Successfully processed and structured {review_id}. Added to list.")

        # --- 파싱 실패한 경우 (DB 적재는 안 되지만 로깅/디버깅에 유용) ---
        else:
             print(f"--> Failed to parse/structure {review_id} (Result: {validation_result}). Skipping storage for this review.")
             # 필요시 실패한 원본/추출 텍스트, 에러 정보 등을 별도로 저장/로깅할 수 있습니다.
             # failed_data = {
             #     "review_id": review_id,
             #     "original_review_text": review,
             #     "raw_model_output": model_output_text,
             #     "validation_result": validation_result,
             #     "processing_timestamp": pd.Timestamp.now()
             # }
             # failed_reviews_list.append(failed_data)


    except Exception as e:
        # 모델 호출 또는 전체 처리 과정 중 발생한 예상치 못한 오류
        print(f"ERROR: An unexpected error occurred during processing {review_id}: {e}")
        # 에러 정보 로깅 등
        # error_data = {
        #     "review_id": review_id,
        #     "original_review_text": review,
        #     "error_message": str(e),
        #     "processing_timestamp": pd.Timestamp.now()
        # }
        # error_list.append(error_data)


    print("-" * (25 + len(review_id) + len(str(len(sample_reviews)))) + "\n")


print(f"Finished processing all sample reviews. Successfully structured {len(processed_reviews_list)} reviews.")
# print(f"Failed to parse/structure {len(sample_reviews) - len(processed_reviews_list)} reviews.")


# --- 결과를 Pandas DataFrame으로 변환 ---
df_processed_reviews = pd.DataFrame(processed_reviews_list)

if not df_processed_reviews.empty:
    print("\n--- Processed Data (DataFrame Head) ---")
    print(df_processed_reviews.head())

    print("\n--- DataFrame Info ---")
    df_processed_reviews.info()

    # --- DataFrame을 CSV 파일로 저장 (DB 적재용) ---
    csv_output_path = "processed_snatch_reviews.csv"
    try:
        df_processed_reviews.to_csv(csv_output_path, index=False, encoding='utf-8')
        print(f"\nSuccessfully saved processed data to CSV file: {csv_output_path}")
        print("This CSV file can now be easily imported into most databases or spreadsheet software.")
    except Exception as e:
        print(f"ERROR: Failed to save DataFrame to CSV: {e}")

    # --- (선택) NoSQL DB 등을 위한 JSONL 형식 저장 ---
    # jsonl_output_path = "processed_snatch_reviews.jsonl"
    # try:
    #     # orient='records'와 lines=True는 각 행을 JSON 객체로, 줄바꿈으로 구분하여 저장
    #     df_processed_reviews.to_json(jsonl_output_path, orient='records', lines=True, force_ascii=False)
    #     print(f"\nSuccessfully saved processed data to JSONL file: {jsonl_output_path}")
    #     print("This JSONL file is suitable for document databases like MongoDB, Firestore, or data lakes.")
    # except Exception as e:
    #     print(f"ERROR: Failed to save DataFrame to JSONL: {e}")

else:
    print("\nNo reviews were successfully processed into the DataFrame.")

Using fine-tuned model: tunedModels/snatchreviewjsonconverterv1-sbydql18q0f2

Processing 6 sample reviews and structuring results...
--- [Processing review_1/6] ---
--> Successfully processed and structured review_1. Added to list.
----------------------------------

--- [Processing review_2/6] ---
--> Successfully processed and structured review_2. Added to list.
----------------------------------

--- [Processing review_3/6] ---
ERROR: An unexpected error occurred during processing review_3: 'NoneType' object has no attribute 'get'
----------------------------------

--- [Processing review_4/6] ---
ERROR: An unexpected error occurred during processing review_4: 'NoneType' object has no attribute 'get'
----------------------------------

--- [Processing review_5/6] ---
--> Failed to parse/structure review_5 (Result: FAILED (Extracted part invalid)). Skipping storage for this review.
----------------------------------

--- [Processing review_6/6] ---
ERROR: An unexpected error occurred